In [1]:
from ipywidgets import interact
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from pandas_datareader import data, wb
import ipywidgets as widgets
from plotly.offline import iplot
from IPython.display import set_matplotlib_formats
import chart_studio


%matplotlib inline
set_matplotlib_formats('svg')
sns.set()
sns.set_palette('Set3')

/Users/Kasper/anaconda3/envs/corona_env_2/lib/python3.8/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def square(x):
    return x * x

In [3]:
interact(square, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

In [4]:
interact(square, x=(0, 100, 10));

interactive(children=(IntSlider(value=50, description='x', step=10), Output()), _dom_classes=('widget-interact…

In [5]:
def friends(name, number):
    return '{} has {} friends!'.format(name, number)

In [6]:
interact(friends, name='Sam', number=(5, 10));

interactive(children=(Text(value='Sam', description='name'), IntSlider(value=7, description='number', max=10, …

In [7]:
interact(friends, name='Sam', number={'One': 1, 'Five': 5, 'Ten': 10});

interactive(children=(Text(value='Sam', description='name'), Dropdown(description='number', options={'One': 1,…

In [8]:
def read_tab(sub):
    """
        ARGS 
            sub must be one of the sub-tables under api.covid19data.dk
        RETURNS
            Pandas dataframe 
    """
    r = requests.get('https://api.covid19data.dk/' + sub)
    return pd.read_json(r.content)

# download Corona data
john_hopkins_df          = read_tab('john_hopkins_data')
ssi_cases_df             = read_tab('ssi_cases')
ssi_cases_ts_df          = read_tab('ssi_cases_ts')
ssi_countries_df         = read_tab('ssi_countries')
ssi_deaths_ts_df         = read_tab('ssi_deaths_ts')
ssi_demo_cases_df        = read_tab('ssi_demo_cases')
ssi_demo_tests_df        = read_tab('ssi_demo_tests')
ssi_geography_cases_df   = read_tab('ssi_geography_cases')
ssi_geography_labs_df    = read_tab('ssi_geography_labs')
ssi_geography_lut_df     = read_tab('ssi_geography_lut')
ssi_hospitalized_df      = read_tab('ssi_hospitalized')
ssi_sources_df           = read_tab('ssi_sources')
worldometers_overview_df = read_tab('worldometers_overview')
worldometers_tests_df    = read_tab('worldometers_tests')

In [9]:
# download population data straight from world bank using pandas-datareader
countries = ['AF', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AG', 'AR', 'AM', 'AW', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BA', 'BW', 'BR', 'BG', 'BF', 'BI', 'CV', 'CM', 'CA', 'KY', 'CF', 'TD', 'CL', 'CN', 'CO', 'KM', 'CD', 'CG', 'CR', 'HR', 'CU', 'CW', 'CY', 'CZ', 'CI', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE', 'SZ', 'ET', 'FO', 'FJ', 'FI', 'FR', 'PF', 'GA', 'GM', 'GE', 'GI', 'GR', 'GL', 'GD', 'GU', 'GT', 'GN', 'GW', 'GY', 'HT', 'HN', 'HK', 'HU', 'IS', 'IN', 'ID', 'IR', 'IQ', 'IE', 'IM', 'IL', 'IT', 'JM', 'JP', 'JO', 'KZ', 'KE', 'KI', 'KP', 'KR', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU', 'MO', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 'MH', 'MR', 'MU', 'MX', 'FM', 'MD', 'MC', 'MN', 'ME', 'MA', 'MZ', 'MM', 'NA', 'NR', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE', 'NG', 'MP', 'NO', 'OM', 'PK', 'PW', 'PS', 'PA', 'PG', 'PY', 'PE', 'PH', 'PL', 'PT', 'PR', 'QA', 'MK', 'RO', 'RU', 'RW', 'KN', 'LC', 'MF', 'VC', 'WS', 'SM', 'ST', 'SA', 'SN', 'RS', 'SC', 'SL', 'SG', 'SX', 'SK', 'SI', 'SB', 'SO', 'ZA', 'SS', 'ES', 'LK', 'SD', 'SR', 'SE', 'CH', 'SY', 'TJ', 'TZ', 'TH', 'TL', 'TG', 'TO', 'TT', 'TN', 'TR', 'TM', 'TC', 'TV', 'UG', 'UA', 'AE', 'GB', 'US', 'UY', 'UZ', 'VU', 'VE', 'VN', 'VG', 'VI', 'YE', 'ZM', 'ZW']
population_df = wb.download(indicator='sp.pop.totl', country=countries, start=2018, end=2018)
# nicify 
population_df = population_df.rename(columns={'sp.pop.totl':'population'}).reset_index().drop('year', 1)

In [10]:
# To date_time and rename columns
john_hopkins_df['dates'] = pd.to_datetime(john_hopkins_df['dates'])
john_hopkins_df.rename(columns={'country_region': 'country', 'province_state': 'state'}, inplace=True)

# Merge population data for each country to the data frame
john_hopkins_df = pd.merge(john_hopkins_df, population_df, on='country', how='left')
john_hopkins_df.sort_values(['country', 'state', 'dates'], inplace=True)

# New variables
john_hopkins_df['deaths_per_mio'] = john_hopkins_df['deaths'] / (john_hopkins_df['population'] / 1000000)

# Rename US to United States
john_hopkins_df.loc[john_hopkins_df['country'] == 'US', 'country'] = 'United States'

# Remove states state specifik rows from the dataframe
#print(f'Dropping the {len(john_hopkins_df[~john_hopkins_df["state"].isnull()])} state specifik rows out of the total {john_hopkins_df.shape[0]} rows of the dataframe')
#john_hopkins_df = john_hopkins_df[john_hopkins_df['state'].isnull()]

In [11]:
# Interactive visualisation
def interactive_plot(country='Denmark', y_var_chosen='Deaths'):
    if y_var_chosen == 'Deaths':
        y_var = 'deaths'
    elif y_var_chosen == 'Deaths per mio. capita':
        y_var = 'deaths_per_mio'
    elif y_var_chosen == 'Confirmed cases':
        y_var = 'confirmed'
    elif y_var_chosen == 'Recovered patients':
        y_var = 'recovered'

    country_chosen = john_hopkins_df.query(f'country == "{country}"')
    days_with_deaths = (country_chosen['deaths'] > 0).sum()
    first_death_date = country_chosen[(country_chosen['deaths'] > 0)]['dates'].min().date()
    print(f'{country} has had a positive death count for {days_with_deaths} days, starting on {first_death_date}')

    above_one = country_chosen[y_var] > 0
    
    ax = country_chosen[above_one].plot(x='dates', y=y_var, marker='o', label=country, figsize=(12,8))
    plt.ylabel(y_var_chosen)
    plt.xlabel('Dates')
    sns.despine()

# Series of all contries in the dataframe to choose from
countries = john_hopkins_df['country'].unique()    

# Make the plot interactive
fig = widgets.interact(interactive_plot,
    country=widgets.Dropdown(description='Country', options=countries, value='Denmark'),
    y_var_chosen=widgets.Dropdown(description='Y-variable',options=['Deaths', 'Deaths per mio. capita', 'Confirmed cases', 'Recovered patients'], value='Deaths')
)

plt.show()

interactive(children=(Dropdown(description='Country', index=46, options=('Afghanistan', 'Albania', 'Algeria', …